In [2]:
# Chargement des bibliothèques
# Installation des packages manquants
if (!requireNamespace("kernlab", quietly = TRUE)) {
  install.packages("kernlab")
}
if (!requireNamespace("tidymodels", quietly = TRUE)) {
  install.packages("tidymodels")
}
if (!requireNamespace("ranger", quietly = TRUE)) {
  install.packages("ranger")
}
if (!requireNamespace("xgboost", quietly = TRUE)) {
  install.packages("xgboost")
}
if (!requireNamespace("earth", quietly = TRUE)) {
  install.packages("earth")
}
if (!requireNamespace("nnet", quietly = TRUE)) {
  install.packages("nnet")
}
if (!requireNamespace("kknn", quietly = TRUE)) {
  install.packages("kknn")
}
if (!requireNamespace("shapviz", quietly = TRUE)) {
  install.packages("shapviz")
}
if (!requireNamespace("DALEX", quietly = TRUE)) {
  install.packages("DALEX")
}
if (!requireNamespace("bonsai", quietly = TRUE)) {
  install.packages("bonsai")
}

if (!requireNamespace("doParallel", quietly = TRUE)) {
  install.packages("doParallel")
}

# Chargement des bibliothèques
library(tidymodels)
library(kernlab)
library(ranger)
library(xgboost)
library(earth)
library(nnet)
library(kknn)
library(shapviz)
library(DALEX)
library(bonsai)
library(parallel)
library(doParallel)

# Downloading packages -------------------------------------------------------
- Downloading kernlab from CRAN ...             OK [2.1 Mb in 0.61s]
Successfully downloaded 1 package in 1.4 seconds.

The following package(s) will be installed:
- kernlab [0.9-32]
These packages will be installed into "~/renv/library/linux-ubuntu-jammy/R-4.4/x86_64-pc-linux-gnu".

# Installing packages --------------------------------------------------------
- Installing kernlab ...                        OK [installed binary and cached in 1.0s]
# Downloading packages -------------------------------------------------------
- Downloading earth from CRAN ...               OK [2 Mb in 0.44s]
- Downloading plotmo from CRAN ...              OK [1.5 Mb in 0.4s]
- Downloading plotrix from CRAN ...             OK [1.1 Mb in 0.43s]
Successfully downloaded 3 packages in 1.8 seconds.

The following package(s) will be installed:
- earth   [5.3.3]
- plotmo  [3.6.3]
- plotrix [3.8-4]
These packages will be installed in

── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.6      ✔ recipes      1.0.10
✔ dials        1.2.1      ✔ rsample      1.2.1 
✔ dplyr        1.1.4      ✔ tibble       3.2.1 
✔ ggplot2      3.5.1      ✔ tidyr        1.3.1 
✔ infer        1.0.7      ✔ tune         1.2.1 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.1      ✔ workflowsets 1.1.0 
✔ purrr        1.0.2      ✔ yardstick    1.3.1 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org


Attaching package: ‘kernlab’


The following object is masked from ‘package:purrr’:

    cross


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:scales’:

    al

In [3]:
stations_data <- read.csv(file = "stations_clim_data.csv")#%>%
#filter(stations=="boromo")
dim(stations_data)

[1] 98622     8

In [4]:
unique(stations_data$stations)

[1] "bobo"       "boromo"     "dori"       "dedougou"   "fada"      
[6] "gaoua"      "ouaga"      "ouahigouya" "po"

In [5]:
head(stations_data)

,Date,stations,tx,tn,rh,rs,ws,et0
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1988-01-01,bobo,29.5,15.0,32,15.70637,1.72,4.461520
2,1988-01-02,bobo,31.4,17.5,27,15.64143,1.67,4.731423
3,1988-01-03,bobo,27.5,17.8,22,13.36923,2.11,4.928952
4,1988-01-04,bobo,27.5,15.0,22,15.92401,1.89,4.762524
5,1988-01-05,bobo,29.3,14.4,24,15.28877,1.11,3.817034
6,1988-01-06,bobo,29.5,16.5,26,14.89933,0.78,3.357057


Modèle en boucle

In [6]:
# Obtenir les noms des colonnes de prédicteurs
predictors <- colnames(stations_data)[-c(1,2,8)]

In [7]:
stations_names=unique(stations_data$stations)

In [8]:
# Boucle sur chaque station
i=0
ns=length(stations_names)
for (station in stations_names) {
  i=i+1
  cat("Processing station:", station, " ( ", i,"/",ns," )\n" )
  
  # Filtrer les données pour la station courante
  station_data <- stations_data %>%
    filter(stations == station)
  
  # Division des données en ensembles d'entraînement et de test
  set.seed(123)
  data_split <- initial_split(station_data, prop = 0.8)
  train_data <- training(data_split)
  test_data <- testing(data_split)
  
  # Prétraitement des données
  recipe <- recipe(et0 ~ tx + tn + rh + rs + ws, data = train_data) %>%
    step_normalize(all_predictors())
  
  # Définir le modèle KNN
  knn_model <- nearest_neighbor(
    neighbors = tune(),
    weight_func = tune()
  ) %>%
    set_engine("kknn") %>%
    set_mode("regression")
  
  # Créer le workflow
  knn_workflow <- workflow() %>%
    add_recipe(recipe) %>%
    add_model(knn_model)
  
  # Définir la grille de recherche aléatoire pour les hyperparamètres
  set.seed(123)
  # Finalize the parameters
  knn_grid <- grid_regular(neighbors(), weight_func(), levels = 10)
  
  cat("Grid length:", nrow(knn_grid), "\n")
  
  # Validation croisée
  cv_folds <- vfold_cv(train_data, v = 5)
  
  # Entraîner et évaluer le modèle
  knn_fit <- tune_grid(
    knn_workflow,
    resamples = cv_folds,
    grid = knn_grid,
    metrics = metric_set(rmse),
    control = control_grid(save_pred = TRUE, parallel_over = "everything")
  )
  
  # Sélectionner le meilleur modèle
  best_params <- select_best(knn_fit, metric = "rmse")
  
  # Finaliser et ajuster le modèle final
  final_knn <- finalize_workflow(knn_workflow, best_params)
  knn_final_model <- fit(final_knn, data = train_data)
  
  # Sauvegarder le modèle final dans un fichier RDS
  saveRDS(knn_final_model, file = paste0("knn_model_", station, ".rds"))
}

Processing station: bobo  (  1 / 9  )
Grid length: 100 
Processing station: boromo  (  2 / 9  )
Grid length: 100 
Processing station: dori  (  3 / 9  )
Grid length: 100 
Processing station: dedougou  (  4 / 9  )
Grid length: 100 
Processing station: fada  (  5 / 9  )
Grid length: 100 
Processing station: gaoua  (  6 / 9  )
Grid length: 100 
Processing station: ouaga  (  7 / 9  )
Grid length: 100 
Processing station: ouahigouya  (  8 / 9  )
Grid length: 100 
Processing station: po  (  9 / 9  )
Grid length: 100 
